Task on hand: To build a model to predict the emotions in the photos.
Will be utilizing Coumputer vision by creating a Convolutional neural network to train the model.

Steps:
1. Creating Environemnt
2. Importing images
3. Importing the image name for extration of target variable (Emotions)
4. Pre prossesing
5. Spliting data into Test Train
6. Rescaling
7. Building Model
8. Prediting target values and checking the accuracy.
9. Checking the model accuracy by prediting the emotion of a photo taken from internet.



In [35]:
#Creating environment
import numpy as np
import pandas as pd
import keras
import os
from PIL import Image, ImageOps
from keras.models import Sequential
from keras .layers import Dense,Dropout,Flatten,Activation
from keras .layers.convolutional import MaxPooling2D,Conv2D
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import cv2

In [2]:
address = "C:\\Users\\sahil\\Downloads\\Expression Data\\Expression Data"
image_list =[Image.open(os.path.join(address, image)) for image in os.listdir(address)]

In [27]:
filename=os.listdir(address)
data=pd.DataFrame(filename,columns=["filename"])
data['expression']=data['filename'].str.slice(3,5)
data.head(2)

,filename,expression
0,KA.AN1.39.tiff,AN
1,KA.AN2.40.tiff,AN


In [28]:
a={'HA':1,'SA':2,'SU':3,'AN':4,'DI':5,'FE':6,'NE':0}
A={'HA':'Happy','SA': 'Sad', 'SU': 'Surprise','AN':'Angry','DI':'Disgust','FE':'Fear','NE': 'Neutral'}
#b={1:'Happy',2: 'Sad', 3: 'Surprise',4:'Angry',5:'Disgust',6:'Fear',7: 'Neutral'}

In [29]:
data['expression'].replace(a,inplace=True)
Y=np.array(data['expression'])

In [7]:
X=[]
for image in image_list:
    imagelist=np.array(image)
    X.append(imagelist)

In [8]:
X=np.array(X)
X.shape

(213, 256, 256)

In [9]:
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.2,random_state=5)
x_train.max()

255

In [10]:
x_train = x_train/255
x_test = x_test/255
x_train.shape

(170, 256, 256)

In [11]:
x_train = x_train.reshape(-1, 256, 256, 1)
x_test = x_test.reshape(-1, 256, 256, 1)

In [12]:
from keras.utils import to_categorical
x_train.shape

(170, 256, 256, 1)

In [13]:
y_train.shape

(170,)

In [14]:
model = Sequential()

model.add(Conv2D(filters=12, kernel_size=(3,3),input_shape=(256,256,1), activation='relu',))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=24, kernel_size=(3,3),input_shape=(256,256,1), activation='relu',))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=50, kernel_size=(3,3),input_shape=(256,256,1), activation='relu',))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Flatten())


model.add(Dense(128))
model.add(Activation('relu'))

model.add(Dropout(0.5))



model.add(Dense(7, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [15]:
model.fit(x_train,y_train,epochs=60)

Epoch 1/60
6/6 [==============================] - 10s 2s/step - loss: 2.3550 - accuracy: 0.1647
Epoch 2/60
6/6 [==============================] - 9s 1s/step - loss: 1.9333 - accuracy: 0.1824
Epoch 3/60
6/6 [==============================] - 10s 2s/step - loss: 1.9114 - accuracy: 0.2647
Epoch 4/60
6/6 [==============================] - 12s 2s/step - loss: 1.8464 - accuracy: 0.3000
Epoch 5/60
6/6 [==============================] - 10s 2s/step - loss: 1.6991 - accuracy: 0.3471
Epoch 6/60
6/6 [==============================] - 10s 2s/step - loss: 1.4948 - accuracy: 0.4588
Epoch 7/60
6/6 [==============================] - 10s 2s/step - loss: 1.3858 - accuracy: 0.5000
Epoch 8/60
6/6 [==============================] - 9s 2s/step - loss: 1.3625 - accuracy: 0.4647
Epoch 9/60
6/6 [==============================] - 10s 2s/step - loss: 1.1119 - accuracy: 0.6176
Epoch 10/60
6/6 [==============================] - 10s 2s/step - loss: 1.0351 - accuracy: 0.6294
Epoch 11/60
6/6 [========================

In [16]:
#Predicting the emotions from the test data
Ypredict=model.predict_classes(x_test)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [17]:
import sklearn.metrics as mt

print("F1 score =",mt.f1_score(y_test,Ypredict,average="weighted"))

F1 score = 0.9297157622739017


In [18]:
print("Performance Metrics",mt.classification_report(y_test,Ypredict))

Performance Metrics               precision    recall  f1-score   support

           0       0.88      1.00      0.93         7
           1       1.00      1.00      1.00         8
           2       1.00      0.83      0.91         6
           3       0.83      1.00      0.91         5
           4       1.00      1.00      1.00         6
           5       1.00      0.80      0.89         5
           6       0.83      0.83      0.83         6

    accuracy                           0.93        43
   macro avg       0.93      0.92      0.92        43
weighted avg       0.94      0.93      0.93        43



In [19]:
Compare=pd.DataFrame([Ypredict,y_test],index=["Predicted Emotion","Actual Data"])
A={1:'Happy',2:'Sad',3:'Surprise',4:'Angry',5:'Disgust',6:'Fear',0: 'Neutral'}
Compare.replace(A,inplace=True)
Compare=Compare.transpose()
Compare["Result"]=""
for j,i in enumerate(Compare["Predicted Emotion"]):
    if Compare["Predicted Emotion"].iloc[j]==Compare["Actual Data"].iloc[j]:
        Compare["Result"].iloc[j]="True"
    else:
        Compare["Result"].iloc[j]="False"
Compare

,Predicted Emotion,Actual Data,Result
0,Surprise,Surprise,True
1,Happy,Happy,True
2,Surprise,Surprise,True
3,Sad,Sad,True
4,Disgust,Disgust,True
5,Happy,Happy,True
6,Sad,Sad,True
7,Disgust,Disgust,True
8,Fear,Fear,True
9,Happy,Happy,True


In [32]:
from skimage import color
from skimage import io
img = color.rgb2gray(io.imread("C:\\Users\\sahil\\Downloads\\New Folder\\image.jpg"))
emotion=cv2.resize(img,(256, 256))
emotion=np.array(emotion)
emotion = emotion.reshape(-1, 256, 256, 1)
sd=model.predict_classes(emotion)
sd

array([5], dtype=int64)